In [3]:
import pandas as pd
import json
import pyarrow.parquet as pq
from pathlib import Path


def read_all_files(folder_path):
    """
    Read all CSV, JSON, and Parquet files from a folder and combine into a single DataFrame.

    Args:
        folder_path: Path to the folder containing the data files

    Returns:
        Combined pandas DataFrame
    """
    folder = Path(folder_path)
    dataframes = []

    # Read all CSV files
    for csv_file in folder.glob("*.csv"):
        print(f"Reading CSV: {csv_file.name}")
        df = pd.read_csv(csv_file)
        dataframes.append(df)

    # Read all JSON files
    for json_file in folder.glob("*.json"):
        print(f"Reading JSON: {json_file.name}")
        try:
            # Try JSON Lines format first (most common for data files)
            tmp_df = pd.read_json(json_file, lines=True)
            dataframes.append(tmp_df)
        except ValueError:
            print(f"Warning: Unexpected JSON structure in {json_file.name}")
        # tmp_df = pd.read_json(json_file, lines=True)
        # dataframes.append(tmp_df)

    # Read all Parquet files
    for parquet_file in folder.glob("*.parquet"):
        print(f"Reading Parquet: {parquet_file.name}")
        table = pq.read_table(parquet_file)
        df = table.to_pandas()
        dataframes.append(df)

    # Combine all dataframes
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df
    else:
        print("No files found!")
        return pd.DataFrame()
    
# Usage
folder_path = "../data/raw/fall2025_L/"
df = read_all_files(folder_path)
print(f"Combined DataFrame shape: {df.shape}")

Reading CSV: ids_2.csv
Reading CSV: ids_0.csv
Reading CSV: ids_1.csv
Reading JSON: ids_7.json
Reading JSON: ids_10.json
Reading JSON: ids_9.json
Reading JSON: ids_4.json
Reading JSON: ids_3.json
Reading Parquet: ids_5.parquet
Reading Parquet: ids_6.parquet
Reading Parquet: ids_11.parquet
Reading Parquet: ids_8.parquet
Combined DataFrame shape: (61128, 79)


In [4]:
# Check if 'labels' column exists
if ' Label' in df.columns:
    print("\nUnique values in 'label' column:")
    print(df[' Label'].unique())

    print("\nValue counts:")
    print(df[' Label'].value_counts())

    print(f"\nTotal unique labels: {df[' Label'].nunique()}")
else:
    print("'labels' column not found in the dataframe")
    print(f"Available columns: {df.columns.tolist()}")


Unique values in 'label' column:
['BENIGN' 'DoS Hulk' 'DoS Slowhttptest' 'Heartbleed' 'DoS GoldenEye']

Value counts:
 Label
DoS Hulk            30027
DoS GoldenEye       20586
DoS Slowhttptest     5499
BENIGN               5005
Heartbleed             11
Name: count, dtype: int64

Total unique labels: 5


In [5]:
# Remove all rows with 'Heartbleed' label
df = df[df[' Label'] != 'Heartbleed']

# Create binary column: 0 for BENIGN, 1 for all attacks
df['Attack'] = (df[' Label'] != 'BENIGN').astype(int)

# Verify the changes
print("\nAfter removing Heartbleed:")
print(df[' Label'].value_counts())

print("\nBinary label distribution:")
print(df['Attack'].value_counts())


After removing Heartbleed:
 Label
DoS Hulk            30027
DoS GoldenEye       20586
DoS Slowhttptest     5499
BENIGN               5005
Name: count, dtype: int64

Binary label distribution:
Attack
1    56112
0     5005
Name: count, dtype: int64


In [6]:
# Save the combined dataframe to CSV
output_path = "../data/interm/combined_raw.csv"

# Create the directory if it doesn't exist
Path(output_path).parent.mkdir(parents=True, exist_ok=True)

# Save to CSV
print(f"\nSaving dataframe to {output_path}...")
df.to_csv(output_path, index=False)
print(
    f"Successfully saved! File size: {Path(output_path).stat().st_size / (1024*1024):.2f} MB")


Saving dataframe to ../data/interm/combined_raw.csv...
Successfully saved! File size: 22.83 MB


11/28/2025 - Nafisa - Load all raw data files
**Purpose:**  
Load all CSV, JSON, and Parquet files from the fall2025_L directory, merge them into a single DataFrame, clean labels, and save the combined dataset for downstream processing.


**Interpretation / Findings:** 
- Total 5 unique labels found: BENIGN, DoS Hulk, DoS GoldenEye, DoS Slowhttptest, Heartbleed.
- Heartbleed (11 rows) removed as required.
- Created binary target column Attack → 1 for attacks, 0 for benign.
- Final dataset size after cleaning:
    -  Attack = 1: 56,112 rows
    - Attack = 0: 5,005 rows
- Output saved to: data/interm/combined_raw.csv

**Notes for Team:**
The combined dataset is now ready for feature engineering and model training. Please ensure to review the cleaned labels and the binary target column for consistency before proceeding. 

### Done with Task

In [1]:
import pandas as pd

# read in the dataset 
df = pd.read_csv('../data/interm/combined_raw.csv')

# Count each label amount 
label_counts = df[' Label'].value_counts()
print("Label Counts:")
print(label_counts)
print()

# Count the total amount of DOS attacks 
total_dos = label_counts[label_counts.index != 'BENIGN'].sum()
print(f"Total DOS attacks: {total_dos}")
print(f"Total BENIGN: {label_counts['BENIGN']}")
print(f"Total records: {len(df)}")

Label Counts:
 Label
DoS Hulk            30027
DoS GoldenEye       20586
DoS Slowhttptest     5499
BENIGN               5005
Name: count, dtype: int64

Total DOS attacks: 56112
Total BENIGN: 5005
Total records: 61117


### Label Distribution

| Label | Count |
|-------|-------|
| DoS Hulk | 30,027 |
| DoS GoldenEye | 20,586 |
| DoS Slowhttptest | 5,499 |
| BENIGN | 5,005 |

**Summary:**
- Total DOS attacks: 56,112
- Total BENIGN: 5,005
- Total records: 61,117

DONE WITH TASK 

In [7]:
print("All columns before formatting:")
print(df.columns.tolist())

All columns before formatting:
[' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',

In [8]:
# Clean column names: remove leading/trailing spaces and replace spaces with underscores
df.columns = df.columns.str.strip().str.replace(' ', '_')

print("All columns after formatting:")
print(df.columns.tolist())

All columns after formatting:
['Destination_Port', 'Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean', 'Bwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length', 'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s', 'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count', 'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count', 'URG_Flag_Count', 'CW

In [9]:
# Find and drop all columns with constant values (including all zeros)
constant_columns = [col for col in df.columns if df[col].nunique() == 1]

print(f"Found {len(constant_columns)} constant columns:")
for col in constant_columns:
    print(f"  {col} = {df[col].iloc[0]}")

# Drop all constant columns
df = df.drop(columns=constant_columns)
print(f"\nShape: {df.shape}")

Found 10 constant columns:
  Bwd_PSH_Flags = 0
  Fwd_URG_Flags = 0
  Bwd_URG_Flags = 0
  CWE_Flag_Count = 0
  Fwd_Avg_Bytes/Bulk = 0
  Fwd_Avg_Packets/Bulk = 0
  Fwd_Avg_Bulk_Rate = 0
  Bwd_Avg_Bytes/Bulk = 0
  Bwd_Avg_Packets/Bulk = 0
  Bwd_Avg_Bulk_Rate = 0

Shape: (61117, 70)


In [10]:
#Checking if these columns have duplicate data 
print((df['Fwd_Header_Length'] == df['Fwd_Header_Length.1']).all())


True


In [11]:
df = df.drop(columns=['Fwd_Header_Length.1'])
print(f"Shape: {df.shape}")

Shape: (61117, 69)


In [12]:
# Identify columns with all NaN values
# Since there are no such columns, this will print an empty list
all_nan_columns = df.columns[df.isna().all()].tolist()

print(f"Columns with all NaN values: {all_nan_columns}")

# Drop rows with any NaN values
df = df.dropna(axis=0, how='any')

print(f"Shape: {df.shape}")

Columns with all NaN values: []
Shape: (61006, 69)


In [14]:
# Save cleaned data
output_path = "../data/cleaned/wednesday_cleaned.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Saved to {output_path} - Shape: {df.shape}")

Saved to ../data/cleaned/wednesday_cleaned.csv - Shape: (61006, 69)


11/29/2025 - Nafisa - Clean Dataset

**Purpose:**  
Clean the raw Wednesday network traffic dataset by standardizing column names, removing irrelevant features, filtering malicious traffic types, and preparing data for machine learning models.

**Interpretation / Findings:** 
- **Original dataset:** 61,117 rows × 80 columns
- **Standardized column names:** Removed leading/trailing spaces, replaced spaces with underscores
- **Removed 10 constant columns** (all zeros): Bwd_PSH_Flags, Fwd_URG_Flags, Bwd_URG_Flags, CWE_Flag_Count, Fwd_Avg_Bytes/Bulk, Fwd_Avg_Packets/Bulk, Fwd_Avg_Bulk_Rate, Bwd_Avg_Bytes/Bulk, Bwd_Avg_Packets/Bulk, Bwd_Avg_Bulk_Rate
- **Removed 1 duplicate column:** Fwd_Header_Length.1 (identical to Fwd_Header_Length)
- **Removed 111 rows** containing NaN values to ensure complete data for ML
- **Cleaned dataset:** 61,006 rows × 69 columns

**Notes for Team:**
- All features are now numeric except Label column
- Saved to: `data/cleaned/wednesday_cleaned.csv`

### Done with Task